In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import optuna

from simulate.simulate import simulate

from inference.loss_minibatch import LossMinibatch
from inference.train_minibatch import train_minibatch

from plot.plot_hist_loss import plot_loss
from plot.plot_posterior import plot_posterior

In [9]:
%config InlineBackend.figure_format = "retina"
sns.set()

In [10]:
data, truth = simulate(use_torch=True)

In [11]:
def objective(trial):
    
    try:
        kwargs = dict(
            n_epochs=200,
            initial_lr=trial.suggest_float("lr", 1e-5, 1e-1, log=True),
            optimizer_name="RMSprop", # trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
            constant_lr=False,        # trial.suggest_categorical("constant_lr", [True, False]),
            flow_length=trial.suggest_int("flow_length", 1, 40),
            n_sample=trial.suggest_int("n_sample", 1, 100),
            batch_size=1000
        )

        z_flow, theta_flow, hist_loss = train_minibatch(data, **kwargs)

        loss = LossMinibatch()(
            batch_size=None, # Take all data at once
            z_flow=z_flow,
            theta_flow=theta_flow,
            n_sample=100,
            n_u=len(np.unique(data['u'])), 
            n_w=len(np.unique(data['w'])),
            **data).item()
    except:
        loss = np.nan
    return loss

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=300)

study.best_params

[I 2021-06-02 15:33:59,926] A new study created in memory with name: no-name-65780f9a-104d-4a1d-bbfd-0e2ace6837a4


  0%|          | 0/200 [00:00<?, ?it/s]

[W 2021-06-02 15:39:46,822] Trial 0 failed, because the objective function returned nan.


  0%|          | 0/200 [00:00<?, ?it/s]